## 一、str对象
### 1. str对象的设计意图
str 对象是定义在 Index 或 Series 上的属性，专门用于逐元素处理文本内容，其内部定义了大量方法，因此对一个序列进行文本处理，首先需要获取其 str 对象。在Python标准库中也有 str 模块，为了使用上的便利，有许多函数的用法 pandas 照搬了它的设计，例如字母转为大写的操作：

In [10]:
import pandas as pd
import numpy as np

str1 = 'abcd'
str.upper(str1)

'ABCD'

In [11]:
s = pd.Series(['abcd', 'ef', 'gh'])
s.str

In [12]:
s.str.upper()

0    ABCD
1      EF
2      GH
dtype: object

### 3 2. 2. []索引器
对于 str 对象而言，可理解为其对字符串进行了序列化的操作，例如在一般的字符串中，通过 [] 可以取出某个位置的元素：

In [13]:
print(str1[0])
print(str1[:-1]) # 切片

a
abc


In [14]:
s

0    abcd
1      ef
2      gh
dtype: object

In [15]:
s.str[0] # 取出每个元素的第一个元素

0    a
1    e
2    g
dtype: object

In [16]:
s.str[:-1]

0    abc
1      e
2      g
dtype: object

### 3. string类型
在上一章提到，从 pandas 的 1.0.0 版本开始，引入了 string 类型，其引入的动机在于：
1. 原来所有的字符串类型都会以 object 类型的 Series 进行存储，
2. 但 object 类型只应当存储混合类型，例如同时存储浮点、字符串、字典、列表、自定义类型等，
3. 因此字符串有必要同数值型或 category 一样，具有自己的数据存放类型，从而引入了 string 类型。

## 二. 正则表达式基础

- 正则表达式是一种按照某种正则模式，从左到右匹配字符串中内容的一种工具。
- 在使用时需要先导入re模块
- 下面使用python 中 re 模块的 findall 函数来匹配所有出现过但不重叠的模式，第一个参数是正则表达式，第二个参数是待匹配的字符串。

In [17]:
import re
re.findall(r'Apple', 'Apple! This Is an Apple!')

['Apple', 'Apple']

 - 几条特殊原则
   - 正则表达式通常都包含反斜杠“\”，可以使用原始字符串来表示它们。模式元素(如 r'\t'，等价于 '\\t')匹配相应的特殊字符
   - 元字符
     - '. ': 匹配除换行符以外的任意字符
     - '*' : 匹配前面的子表达式零次或多次
     - '+ ' ： 匹配前面的子表达式一次或多次
     - ? : 匹配前面的子表达式零次或一次
     - []: 字符类，匹配方括号中包含的任意字符
     - {n,m} : 花括号，匹配前面字符至少 n 次，但是不超过 m 次
     - (xyz): 字符组，按照确切的顺序匹配字符xyz

 1. ^上尖符号表示以后面的字开头，$美元符号表示以后面的字符结尾

In [18]:
str_1 = 'bbccpython223234xky'
re1 = '^b.*'  # 以b开头，匹配后面任意字符
re2 = '.*p$'
res1 = re.match(re1, str_1)
res2 = re.match(re2, str_1)
res3 = re.match(re2, 'loop')
print(res1)
print(res2)
print(res3)

<re.Match object; span=(0, 19), match='bbccpython223234xky'>
None
<re.Match object; span=(0, 4), match='loop'>


 2. "?"非贪婪模式，从左边开始匹配。没有?号为贪婪模式

In [19]:
new_str = 'acbooobbyabc' #提取中间的booob
re_str1 = '.*(b.*b).*' #b不会成功，贪婪模式，从右边开始，匹配到byab
re_str2 = '.*?(b.*b).*'  #不会成功，左边非贪婪，但是右边为贪婪模式，从右边开始，匹配到booobbyab
re_str3 = '.*?(b.*?b).*' #成功

print(re.match(re_str1, new_str).group(1))
print(re.match(re_str2, new_str).group(1))
print(re.match(re_str3, new_str).group(1))

byab
booobbyab
booob


 3. '+'表示前面的字符至少出现一次

In [20]:
str5 = 'boooooobbbbo'
re_str5 = '.*(b.*b).*'  #贪婪模式，匹配到bb，因为bb中间的".*" -任意字符0或者任意次
re_str6 = '.*(b.+b).*'  #贪婪模式，匹配到bbb，因为bb中间的".+" -任意字符1次或者任意次（至少一次）
print(re.match(re_str5, str5).group(1))
print(re.match(re_str6, str5).group(1))

bb
bbb


4. {}的用法 {2},{2，}，{2，5}

In [21]:
line = 'booobybbobbbo'
re_str1 = '.*(bo{3})' # 匹配{}前面的字符n次，n为括号内的数
re_str2 = '.*(^b.{3,})' #大于等于n次 "o{1,}" 等价于 "o+"。"o{0,}" 则等价于 "o*"
print(re.match(re_str1, line).group(1))
print(re.match(re_str2, line).group(1))

booo
booobybbobbbo


 5. '[]'的3种用法
5.1 [abcd], 匹配abcd中的任意一个字符
5.2[^abcd], 匹配除了bcd中的任意一个字符
5.3 [A-D], [0-9],[a-z],区间任意个字符 [A-Za-z0-9]表示任意字母和数字

 6. 简写字符集
 -  6.1 - \w 匹配所有字母、数字、下划线: [a-zA-Z0-9_]
 -  6.2- \W 匹配非字母和数字的字符: [^\w]
 -  6.3- \d 匹配数字: [0-9]
 -  6.4- \D 匹配非数字: [^\d]
 -  6.5- \s 匹配空格符: [\t\n\f\r\p{Z}]
 -  6.6- \S 匹配非空格符: [^\s]
 -  6.7- \B 匹配一组非空字符开头或结尾的位置，不代表具体字符

## 二. str对象常用的方法

### 2.1 split方法 str.split

In [22]:
s = '上海市/黄浦区/方浜中路/249号'
s.split('/')

['上海市', '黄浦区', '方浜中路', '249号']

### 2.2 合并，join

In [23]:
# 举例： 一句代码 str($125,000) to int(125000) 
money = '$125,000'
number = int(''.join(money.split('$')[1].split(',')))
print(money)
print(number)

$125,000
125000


### 2.3 匹配


- str.contains： 用于判断每个字符串是否包含正则表达式的字符|
- str.startswith：用于判断是否以字符串为开始|
- str.endswith：用于判断是否以字符串为结束|
- str.match：每个字符串起始处是否符合给定正则表达式的字符|
- str.find：从左到右第一次匹配的位置的索引|
- str.rfind：从右到左第一次匹配的位置的索引|

In [24]:
s = pd.Series(['my cat', 'he is fat', 'railway station'])
s.str.contains('\s\wat')

0     True
1     True
2    False
dtype: bool

In [25]:
s.str.match('m|h')

0     True
1     True
2    False
dtype: bool

### 2.4 替换
str.replace 和 replace 并不是一个函数，在使用字符串替换时应当使用前者。

In [26]:
s = pd.Series(['a_1_b','c_?'])
s.str.replace('\d|\?', 'new', regex=True)

0    a_new_b
1      c_new
dtype: object

### 2.5 提取
提取既可以认为是一种返回具体元素（而不是布尔值或元素对应的索引位置）的匹配操作，也可以认为是一种特殊的拆分操作。前面提到的 str.split 例子中会把分隔符去除，这并不是用户想要的效果，这时候就可以用 str.extract 进行提取

## 练习

In [30]:
# 房屋信息数据集
df = pd.read_excel('house_info.xls', 
                   usecols=['floor','year','area','price'])
df.head()

,floor,year,area,price
0,高层（共6层）,1986年建,58.23㎡,155万
1,中层（共20层）,2020年建,88㎡,155万
2,低层（共28层）,2010年建,89.33㎡,365万
3,低层（共20层）,2014年建,82㎡,308万
4,高层（共1层）,2015年建,98㎡,117万


In [32]:
# 1. 将year列改成整数年份存储
df['year'] = df['year'].str[:4]
df.head()

,floor,year,area,price
0,高层（共6层）,1986,58.23㎡,155万
1,中层（共20层）,2020,88㎡,155万
2,低层（共28层）,2010,89.33㎡,365万
3,低层（共20层）,2014,82㎡,308万
4,高层（共1层）,2015,98㎡,117万


In [37]:
# 2. 将floor列替换为Level, Highest两列，
# 其中的元素分别为string类型的层类别（高层、中层、低层）与整数类型的最高层数。
pat = '(?P<Level>\w层)（共(?P<Highest>\d+)层）'
re.findall(pat, df.floor[0]) # 测试

[('高层', '6')]

In [38]:
df['floor'].str.extract(pat).head(10)

,Level,Highest
0,高层,6
1,中层,20
2,低层,28
3,低层,20
4,高层,1
5,中层,21
6,低层,14
7,中层,6
8,高层,1
9,低层,33


In [39]:
# 进行数据列连接
df = pd.concat([df.drop(columns='floor'), df.floor.str.extract(pat)], axis=1)

df.Level = df.Level.astype('string')
df.Highest = pd.to_numeric(df.Highest).astype('Int64')

In [40]:
df.head(3)

,year,area,price,Level,Highest
0,1986,58.23㎡,155万,高层,6
1,2020,88㎡,155万,中层,20
2,2010,89.33㎡,365万,低层,28


In [41]:
# 3. 计算房屋每平米的均价avg_price，以***元/平米的格式存储到表中，其中***为整数。
df.head(3)

,year,area,price,Level,Highest
0,1986,58.23㎡,155万,高层,6
1,2020,88㎡,155万,中层,20
2,2010,89.33㎡,365万,低层,28


In [42]:
area = pd.to_numeric(df.area.str[:-1])
price = pd.to_numeric(df.price.str[:-1])
print(area[:3])
print(price[:3])

0    58.23
1    88.00
2    89.33
Name: area, dtype: float64
0    155.0
1    155.0
2    365.0
Name: price, dtype: float64


In [44]:
df['avg_price'] = (round(price * 10000 / area, 2)).astype('string') + '元/平米'
df.head()

,year,area,price,Level,Highest,avg_price
0,1986,58.23㎡,155万,高层,6,26618.58元/平米
1,2020,88㎡,155万,中层,20,17613.64元/平米
2,2010,89.33㎡,365万,低层,28,40859.73元/平米
3,2014,82㎡,308万,低层,20,37560.98元/平米
4,2015,98㎡,117万,高层,1,11938.78元/平米


### Ex2：《权力的游戏》剧本数据集

In [45]:
# 现有一份权力的游戏剧本数据集如下：
df = pd.read_csv('script.csv')
df.head()

,Release Date,Season,Episode,Episode Title,Name,Sentence
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.


In [49]:
# 1. 计算每一个Episode的台词条数
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23911 entries, 0 to 23910
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Release Date   23911 non-null  object
 1    Season        23911 non-null  object
 2   Episode        23911 non-null  object
 3   Episode Title  23911 non-null  object
 4   Name           23908 non-null  object
 5   Sentence       23911 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


In [51]:
df.columns

Index(['Release Date', ' Season', 'Episode ', 'Episode Title', 'Name',
       'Sentence'],
      dtype='object')

In [50]:
df[' Season'].value_counts()

Season 2    3914
Season 3    3573
Season 4    3446
Season 1    3179
Season 5    3035
Season 6    2856
Season 7    2442
Season 8    1466
Name:  Season, dtype: int64

In [52]:
df['Episode '].value_counts()

Episode 5     3083
Episode 2     2957
Episode 3     2648
Episode 1     2637
Episode 7     2447
Episode 6     2380
Episode 4     2356
Episode 8     1880
Episode 10    1846
Episode 9     1677
Name: Episode , dtype: int64

In [53]:
df.groupby([' Season', 'Episode '])['Sentence'].count()

 Season   Episode   
Season 1  Episode 1     327
          Episode 10    266
          Episode 2     283
          Episode 3     353
          Episode 4     404
                       ... 
Season 8  Episode 2     405
          Episode 3     155
          Episode 4      51
          Episode 5     308
          Episode 6     240
Name: Sentence, Length: 73, dtype: int64

In [54]:
# 2. 以空格为单词的分割符号，请求出单句台词平均单词量最多的前五个人。
words_count = df.set_index('Name')['Sentence'].str.split(' ').str.len().groupby('Name').mean()
words_count.sort_values(ascending=False)[:5].index.tolist()

['male singer',
 'slave owner',
 'manderly',
 'lollys stokeworth',
 'dothraki matron']